In [2]:
!pip install tqdm together datasets

In [1]:
import json
import time
import requests
import re
import csv
import together
import os
from tqdm import tqdm
from datasets import Dataset

In [ ]:
TOGETHER_API_KEY = "nottherealkey" # 
MODEL_NAME ="meta-llama/Llama-4-Scout-17B-16E-Instruct" # sau alt model valid
API_URL = "https://api.together.xyz/inference"
together.api_key = TOGETHER_API_KEY

In [9]:
def extract_choices_from_response(text):
    text = text.upper()
    match = re.search(r"RĂSPUNS(?:\s+CORECT)?\s*[:：]?\s*([ABC](?:[\s,]+[ABC])*)", text)
    if match:
        return set(re.findall(r"[ABC]", match.group(1)))
    return set(re.findall(r"\b[ABC]\b", text))

def evaluate_model_together(dataset, model="meta-llama/Llama-3-70b-chat-hf",
                            max_tokens=128,
                            output_csv_path="evaluare_together_output.csv",
                            log_progress=10,
                            verbose=False,
                            index=0,
                            limit=1000):

    golds, preds = [], []
    exact_matches = 0
    client = together.Together(api_key=TOGETHER_API_KEY)

    # Verificăm dacă fișierul CSV deja există
    file_exists = os.path.exists(output_csv_path)

    with open(output_csv_path, "a", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["index", "prompt", "golds", "preds", "response"])
        if not file_exists:
            writer.writeheader()

        for i, ex in enumerate(tqdm(dataset, desc=f"Evaluare {model}", unit="ex")):
            if i < index or i > limit:
                continue
            try:
                # verificam daca exista image
                image_str=""
                if "image" not in ex or not ex["image"]:
                    print(f"Warning: Exercițiul {i} nu are imagine asociată.")
                else:
                    image_str = str(ex.get("image", "")).strip()
                text_str = str(ex.get("text", "")).strip()
                prompt_text = (
                    image_str + "\n"
                    + text_str + "\n"
                    + "Mai întâi oferă doar litera/literele răspunsului corect sub forma:\nRăspuns: A\n"
                    + "Apoi oferă o explicație succintă pe un rând nou. \n"
                    + "Răspuns:"
                )
                prompt = (
                    prompt_text
                )
                gold = extract_choices_from_response(ex.get("response", ""))

                response = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=0.6
                )

                decoded = response.choices[0].message.content.strip()
                pred = extract_choices_from_response(decoded)

                if verbose and (i % log_progress == 0 or i == len(dataset) - 1):
                    print(f"\n📘 Prompt [{i}]:", prompt)
                    print("✅ Gold:", gold)
                    print("🧠 Pred:", pred)
                    print("📝 Decoded:", decoded)
                    print("-" * 60)

                if gold and pred:
                    golds.append(gold)
                    preds.append(pred)
                    if gold == pred:
                        exact_matches += 1

                writer.writerow({
                    "index": i,
                    "prompt": ex["text"],
                    "golds": ",".join(sorted(gold)) if gold else "",
                    "preds": ",".join(sorted(pred)) if pred else "",
                    "response": decoded
                })
                f.flush()

                time.sleep(1)  # Respectă rate limit

            except Exception as e:
                print(f"[❌ EROARE la index {i}] {e}")
                continue

    print(f"\n📝 Evaluare finalizată. CSV salvat la: {output_csv_path}")
    total = len(golds)
    accuracy = exact_matches / total if total else 0
    print(f"📊 Acuratețe totală (exact match): {accuracy:.2%}")
    return golds, preds, accuracy


In [7]:
def load_jsonl_as_list(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

data = load_jsonl_as_list("multimodal.jsonl")
dataset_text_only = Dataset.from_list(data)

split_dataset = dataset_text_only.train_test_split(test_size=0.2, seed=42)
train_ds = split_dataset["train"]
test_ds = split_dataset["test"]
print(test_ds[0])

{'image': None, 'text': 'Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.\nÎntrebare: În care dintre situații depășirea este interzisă?\nA: În intersecțiile cu circulație nedirijată și la trecerile pentru pietoni semnalizate\nB: În intersecții și la o distanță de 50 m de acestea\nC: În zona de acțiune a indicatorului „limitare de viteză“\nPoate exista mai mult de un răspuns corect.', 'response': 'A'}


In [10]:
golds, preds, accuracy = evaluate_model_together(
        model=MODEL_NAME,
        dataset=test_ds,
        max_tokens=128,
        log_progress=10,
        verbose=True,
        output_csv_path="multimodal_rezultate_llama4_scout.csv",
    )

Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct:   0%|          | 0/242 [00:00<?, ?ex/s]


📘 Prompt [0]: 
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: În care dintre situații depășirea este interzisă?
A: În intersecțiile cu circulație nedirijată și la trecerile pentru pietoni semnalizate
B: În intersecții și la o distanță de 50 m de acestea
C: În zona de acțiune a indicatorului „limitare de viteză“
Poate exista mai mult de un răspuns corect.
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou. 
Răspuns:
✅ Gold: {'A'}
🧠 Pred: {'A'}
📝 Decoded: Răspuns: A

Depășirea este interzisă în locurile unde vizibilitatea este redusă sau unde manevra ar putea pune în pericol participanții la trafic, inclusiv în intersecțiile cu circulație nedirijată și la trecerile pentru pietoni semnalizate.
------------------------------------------------------------


Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct:   1%|          | 2/242 [00:06<11:54,  2.98s/ex]

Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct:   2%|▏         | 4/242 [00:13<14:20,  3.62s/ex]

Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct:   2%|▏         | 5/242 [00:16<13:27,  3.41s/ex]

...


📘 Prompt [240]: 
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: Pe drumurile publice prevăzute cu minimum 3 benzi pe sens, atunci când conducătorii a două vehicule circulă în aceeași direcție, dar pe benzi diferite, și intenționează să se înscrie pe banda liberă dintre ei, această bandă va fi ocupată de:
A: Vehiculul care circulă pe banda din stânga, față de cel care circulă pe banda din dreapta
B: Vehiculul care circulă pe banda din dreapta, față de cel care circulă pe banda din stânga
C: Vehiculul care circulă cu viteza maximă admisă pe sectorul de drum respectiv
Poate exista mai mult de un răspuns corect.
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou. 
Răspuns:
✅ Gold: {'A'}
🧠 Pred: {'A'}
📝 Decoded: Răspuns: A

Explicație: În conformitate cu regulile de circulație rutieră, atunci când doi conducători de vehicule circulă în aceeași direcție pe drumuri cu cel puți

Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct: 100%|█████████▉| 241/242 [09:34<00:02,  2.60s/ex]


📘 Prompt [241]: 
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: Oprirea este interzisă:
A: La mai puțin de 25 m de colțul unei intersecții
B: În zona de acțiune a indicatorului „staționarea interzisă“
C: În curbe și pe sectoarele de drum cu vizibilitate redusă sub 50 m
Poate exista mai mult de un răspuns corect.
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou. 
Răspuns:
✅ Gold: {'C', 'A'}
🧠 Pred: {'C', 'B'}
📝 Decoded: Răspuns: B, C

Oprirea este interzisă în zonele indicate de indicatorul "staționarea interzisă" și în curbe sau pe sectoarele de drum cu vizibilitate redusă sub 50 m, pentru a asigura siguranța traficului.
------------------------------------------------------------


Evaluare meta-llama/Llama-4-Scout-17B-16E-Instruct: 100%|██████████| 242/242 [09:37<00:00,  2.38s/ex]


📝 Evaluare finalizată. CSV salvat la: multimodal_rezultate_llama4_scout.csv
📊 Acuratețe totală (exact match): 55.79%
